In [1]:
import chainer
import chainer.functions as F
import chainer.links as L
# trianerクラスが新機能として追加
from chainer import training
from chainer.training import extensions

In [3]:
# 自動的にMNISTデータセットをダウンロードし、NumPy配列を $(HOME)/.chainer ディレクトリに保存する。
train, test = chainer.datasets.get_mnist()

# chainer.iterators.SerialIteratorが内部でchainer.dataset.iterator(以下イテレーター)を呼んでいる
# 訓練データセットは毎ループでシャッフルしたい。
train_iter = chainer.iterators.SerialIterator(train, batch_size=100)

# shuffle=Falseを引数に与えることで、シャッフルを無効化できる
# repeat=Falseとしたが、これはすべての要素を見た時に繰り返しが終了することを意味する。
test_iter = chainer.iterators.SerialIterator(test, 100,repeat=False, shuffle=False)

In [2]:
# ネットワークの組み方・ハイパーパラ、メータの設定は変更なし
class MnistModel(chainer.Chain):
    def __init__(self):
        super(MnistModel,self).__init__(
                l1 = L.Linear(784,100),
                l2 = L.Linear(100,100),
                l3 = L.Linear(100,10))

    def __call__(self,x):    
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        return self.l3(h)

In [ ]:
class Classifier(Chain):
    def __init__(self, predictor):
        super(Classifier, self).__init__(predictor=predictor)

    def __call__(self, x, t):
        y = self.predictor(x)
        loss = F.softmax_cross_entropy(y, t)
        accuracy = F.accuracy(y, t)
        # report()関数は損失と精度の値をtrainerに報告する
        report({'loss': loss, 'accuracy': accuracy}, self)
        return loss

In [ ]:
model = L.Classifier(MnistModel())
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

## Trainingクラス
### Updater
### Trainer
自分たちで書いていた、学習ループやログ出力などを代わりに行ってくれるもの


エポックごとに自動でミニバッチを作ってくれます。

In [4]:
# Set up a Trainer
updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (100, 'epoch'), out="result1")

# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))

# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

# Print selected entries of the log to stdout
# Here "main" refers to the target link of the "main" optimizer again, and
# "validation" refers to the default name of the Evaluator extension.
# Entries other than 'epoch' are reported by the Classifier link, called by
# either the updater or the evaluator.
trainer.extend(extensions.PrintReport( ['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))

# Print a progress bar to stdout
trainer.extend(extensions.ProgressBar())

In [5]:
# Run the training
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
     total [..................................................]  0.33%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 100 epochs
    185.72 iters/sec. Estimated time to finish: 0:05:21.989170.
     total [..................................................]  0.50%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 100 epochs
    192.72 iters/sec. Estimated time to finish: 0:05:09.776790.
     total [..................................................]  0.67%
this epoch [#################################.................] 66.67%
       400 iter, 0 epoch / 100 epochs
    197.15 iters/sec. Estimated time to finish: 0:05:02.303888.
     total [..................................................]  0.83%
this epoch [#########################################.........] 83.33%
       500 iter, 0 epoch / 100 epochs
  

FileExistsError: [WinError 183] 既に存在するファイルを作成することはできません。: 'C:\\Users\\alluser\\Documents\\GitHub\\chainer_course\\result1\\logrdcssky8' -> 'result1\\log'

In [ ]:
#その他　trainerの機能

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot at each epoch
trainer.extend(extensions.snapshot())

if args.resume:
    # Resume from a snapshot
    chainer.serializers.load_npz(args.resume, trainer)